# Playing around with this notebook https://github.com/rowandl/portfolio/blob/master/Webscraping%20Indeed.com/Webscraping%20Indeed.ipynb

# Web Scraping for Indeed.com & Predicting Salaries

In this project, I will practice two major skills: collecting data by scraping a website and then building a binary classifier.

I am going to collect salary information on data science jobs in a variety of markets. Then using the location, title, and summary of the job I will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information, being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

I will convert this problem into classification and use a random forest classifier, as well as another classifier. 

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

I will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Set up a request (using requests) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)
The URL here has many query parameters
- q for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- l for a location
- start for what result number to start on

In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
from urllib import urlopen
import pandas as pd
import matplotlib as plt
%matplotlib inline

In [3]:
## YOUR CODE HERE
html = urllib.urlopen(URL).read()
soup = BeautifulSoup(html, "html.parser")
len(soup)

4

Let's look at one result more closely. A single result looks like
```JSON
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&campaignid=serp-linkcompanyname&fromjk=2480d203f7e97210&jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a nobr element inside of a td element with class='snip.
- The title of a job is in a link with class set to jobtitle and a data-tn-element="jobTitle.
- The location is set in a span with class='location'.
- The company is set in a span with class='company'.

## Functions to extract each item: location, company, job, and salary.¶
Example
```python
def extract_location_from_result(result):
    return result.find ...
```


In [4]:
dflocation = pd.DataFrame(columns=["location"])
dfcompany = pd.DataFrame(columns=["company"])
dfjob_title = pd.DataFrame(columns=["job_title"])
dfsalary = pd.DataFrame(columns=["salary"])

#result.find(‘a’,attrs={‘data-tn-element’:‘jobTitle’}).text
def extract_location(result):
    for b in result.find_all('span', {'class': 'location'}):
        location = b.text
        dflocation.loc[len(dflocation)] = [location]    
        
def extract_company(result):        
    for i in result.find_all('span', {'class':'company'}):
        company = i.text
        dfcompany.loc[len(dfcompany)] = [company]   

def extract_job_title(result):
    for a in result.find_all('a', {'data-tn-element':'jobTitle'}):
        job_title = a.text
        dfjob_title.loc[len(dfjob_title)] = [job_title]

def extract_salary(result):
    for entry in result.find_all('td', {'class' : 'snip'}):
        try:
            salary = entry.find('nobr').text
            dfsalary.loc[len(dfsalary)] = [salary]  
        except:
            salary = 'NA'
            dfsalary.loc[len(dfsalary)] = [salary]    


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.
- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the l=New+York and the start=10. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).


In [139]:
#making a list of cities to run through
cities = ['New+York%2C+NY']

In [140]:
#making a dataframe which collects all the information from my webscraping
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
query = "machine+learning"
url_template= "https://www.indeed.com/jobs?q={}&l={}&start={}"
max_results_per_city = 100 # Set this to a high-value to generate more results. 

df = pd.DataFrame(columns=["location", 'company', 'job_title', 'salary'])
def validator(elem):
    if elem:
        return elem.text
    else:
        return 'NA'

for city in cities:
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(query,city,start)
        html = urllib.urlopen(url).read()
        soups = BeautifulSoup(html, "html.parser")
#         with open('somefile.html', 'a') as the_file:
#             the_file.write(str(soups))
        rows = soups.find_all('div', attrs = {'class':'row'})
        for b in rows:
#             print(b)
            location = validator(b.find('div', attrs = {'class': 'location'}))
            job_title = validator(b.find('a', attrs = {'data-tn-element':'jobTitle'}))
            company = validator(b.find('span', attrs = {'class':'company'}))
            salary = validator(b.find(name="span", attrs={"class":"no-wrap"}))
            df = df.append({"location":location,"company":company, "job_title": job_title, "salary": salary}, ignore_index=True) 
   
df.head()        

,location,company,job_title,salary
0,"New York, NY 10003 (Greenwich Village area)",\n BuildingLink,IoT Specialist,NA
1,"New York, NY 10013 (Tribeca area)",\n\n WeWork,"Manager, Machine Learning",NA
2,"New York, NY",\n\n Owkin,Legal and Business Analyst,"\n $50,000 - $80,000 a year"
3,"New York, NY",\n\n The Hartford,Data Scientist - innovation lab,NA
4,NA,\n Mei Messaging,NLP and Machine Learning - Recent Grad,\n $25 - $35 an hour


In [141]:
#sending it to csvs to save the data
df.to_csv("~/Desktop/machineLearning.csv" , sep=',', encoding='utf-8')

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [143]:
#reading in the data
df1 = pd.read_csv('~/Desktop/machineLearning.csv', index_col=0)
# df2 = pd.read_csv('~/Desktop/GA/April11.csv')
# df3 = pd.read_csv('~/Desktop/GA/April12.csv')
# df4 = pd.read_csv('~/Desktop/GA/April1_11.csv')
# df5 = pd.read_csv('~/Desktop/GA/April2_11.csv')
# df6 = pd.read_csv('~/Desktop/GA/April_10.csv')
# df7 = pd.read_csv('~/Desktop/GA/April_11.csv')

In [144]:
data = pd.concat([df1]) #making into one df
# data.drop(['Unnamed: 0'], axis=1, inplace=True) #resetting index
data.drop_duplicates(inplace=True) #dropping duplicates
data.company.replace(regex=True,inplace=True,to_replace=["\n", "\r"],value="") #getting rid of /n in company
data.salary.replace(regex=True, inplace=True, to_replace=["\n", "\r", "\$"], value="") #getting rid of $ in salary

In [145]:
def eda(dataframe): #code chunk to check quality of data
    print "missing values \n", dataframe.isnull().sum() #shows total amount of null values for each column
    print "dataframe types \n", dataframe.dtypes
    print "dataframe shape \n", dataframe.shape     
    print "dataframe describe \n", dataframe.describe()
    print "dataframe length =", len(dataframe) #length of the dataframe
    print "duplicates", dataframe.duplicated().sum() # added this to duplicates in the data
    for item in dataframe:
        print item
        print dataframe[item].nunique()

eda(data)

missing values 
location     86
company       0
job_title     0
salary       92
dtype: int64
dataframe types 
location     object
company      object
job_title    object
salary       object
dtype: object
dataframe shape 
(115, 4)
dataframe describe 
            location         company                  job_title  \
count             29             115                        115   
unique            15              85                         98   
top     New York, NY          Google  Machine Learning Engineer   
freq              10               9                         11   

                            salary  
count                           23  
unique                          21  
top                     20 an hour  
freq                             3  
dataframe length = 115
duplicates 0
location
15
company
85
job_title
98
salary
21


Lastly, I need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

I didn't think it was safe to multiply data for hour/week/month so I only wanted yearly data.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly, remove duplicate entries

In [146]:
salaries = data[data.salary.notnull()]
salaries = salaries[salaries.salary.str.contains('year')] #only getting yearly salaries
#http://stackoverflow.com/questions/15325182/how-to-filter-rows-in-pandas-by-regex
salaries.describe()

,location,company,job_title,salary
count,7,17,17,17
unique,3,14,17,17
top,"New York, NY",Harnham US,Machine Learning Data Scientist,"120,000 - 170,000 a year"
freq,5,4,1,1


In [147]:
#splitting my salaries to find an average
salaries.salary.replace(regex=True, inplace=True, to_replace="a year", value="")
salaries.salary.replace(regex=True, inplace=True, to_replace=",", value="")
salaries.salary.replace(regex=True, inplace=True, to_replace=" \(Indeed est.\)", value="")
salaries['salary_split'] = salaries['salary'].str.split('-')
salaries.head()

,location,company,job_title,salary,salary_split
2,"New York, NY",Owkin,Legal and Business Analyst,50000 - 80000,"[ 50000 , 80000 ]"
5,NaN,MINDPORTS AI,Quantitative Analysis and Machine Learning,150000 - 185000,"[ 150000 , 185000 ]"
9,NaN,Kasisto,Machine Learning Engineer,140000 - 160000,"[ 140000 , 160000 ]"
17,"New York, NY",Harnham US,C++ Engineer - Computer Vision,130000 - 150000,"[ 130000 , 150000 ]"
19,NaN,untapt,Talent Advocate,50000 - 60000,"[ 50000 , 60000 ]"


In [148]:
#finding an average salary
def avg(salaries):
    salaries['lower'] = salaries['salary_split'].str[0].astype('float')
    salaries['upper'] = salaries['salary_split'].str[1].astype('float')
    salaries['avg'] = salaries[['lower','upper']].mean(axis=1)
avg(salaries)
salaries.head()

,location,company,job_title,salary,salary_split,lower,upper,avg
2,"New York, NY",Owkin,Legal and Business Analyst,50000 - 80000,"[ 50000 , 80000 ]",50000.0,80000.0,65000.0
5,NaN,MINDPORTS AI,Quantitative Analysis and Machine Learning,150000 - 185000,"[ 150000 , 185000 ]",150000.0,185000.0,167500.0
9,NaN,Kasisto,Machine Learning Engineer,140000 - 160000,"[ 140000 , 160000 ]",140000.0,160000.0,150000.0
17,"New York, NY",Harnham US,C++ Engineer - Computer Vision,130000 - 150000,"[ 130000 , 150000 ]",130000.0,150000.0,140000.0
19,NaN,untapt,Talent Advocate,50000 - 60000,"[ 50000 , 60000 ]",50000.0,60000.0,55000.0


In [149]:
#dropping columns so I'm only left with average column
clean_sal = salaries.drop(['salary','salary_split', 'lower', 'upper'], axis=1)
clean_sal.head()

,location,company,job_title,avg
2,"New York, NY",Owkin,Legal and Business Analyst,65000.0
5,NaN,MINDPORTS AI,Quantitative Analysis and Machine Learning,167500.0
9,NaN,Kasisto,Machine Learning Engineer,150000.0
17,"New York, NY",Harnham US,C++ Engineer - Computer Vision,140000.0
19,NaN,untapt,Talent Advocate,55000.0


In [150]:
clean_sal['citystate'] = clean_sal['location'].str.split(',') #splitting the location to separate city and state
clean_sal['city'] = clean_sal['citystate'].str[0] #getting cities
clean_sal['state'] = clean_sal['citystate'].str[1] #getting states
clean_sal['state'] = clean_sal['state'].str[0:3] #getting only 2 letter state codes
clean_sal.drop(['location','citystate'], axis=1, inplace=True) #dropping columns so I'm only left with cities/states
clean_sal.head()

,company,job_title,avg,city,state
2,Owkin,Legal and Business Analyst,65000.0,New York,NY
5,MINDPORTS AI,Quantitative Analysis and Machine Learning,167500.0,NaN,NaN
9,Kasisto,Machine Learning Engineer,150000.0,NaN,NaN
17,Harnham US,C++ Engineer - Computer Vision,140000.0,New York,NY
19,untapt,Talent Advocate,55000.0,NaN,NaN


In [151]:
eda(clean_sal)

missing values 
company       0
job_title     0
avg           0
city         10
state        10
dtype: int64
dataframe types 
company       object
job_title     object
avg          float64
city          object
state         object
dtype: object
dataframe shape 
(17, 5)
dataframe describe 
                 avg
count      17.000000
mean   119495.058824
std     64075.601168
min     36500.000000
25%     60000.000000
50%    130000.000000
75%    150000.000000
max    250000.000000
dataframe length = 17
duplicates 0
company
14
job_title
17
avg
16
city
2
state
1


### Save your results as a CSV

In [152]:
#saving my clean salary data to a csv
clean_sal.to_csv("~/Desktop/clean_salary.csv" , sep=',', encoding='utf-8')

## Predicting salaries using Random Forests + Another Classifier

In [153]:
clean_sal['avg'].describe()

count        17.000000
mean     119495.058824
std       64075.601168
min       36500.000000
25%       60000.000000
50%      130000.000000
75%      150000.000000
max      250000.000000
Name: avg, dtype: float64

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't _have_ to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [154]:
#finding the median
import numpy as np
median = np.median(clean_sal.avg)
print median

130000.0


In [155]:
c = pd.read_csv('~/Desktop/clean_salary.csv', index_col=0)

In [156]:
c['dumsal'] = (c["avg"] >= c["avg"].median()).astype(int)
#http://stackoverflow.com/questions/36637011/how-can-i-create-a-dummy-variable-in-python-with-a-condition-below-or-above-medi
c.head()

,company,job_title,avg,city,state,dumsal
2,Owkin,Legal and Business Analyst,65000.0,New York,NY,0
5,MINDPORTS AI,Quantitative Analysis and Machine Learning,167500.0,NaN,NaN,1
9,Kasisto,Machine Learning Engineer,150000.0,NaN,NaN,1
17,Harnham US,C++ Engineer - Computer Vision,140000.0,New York,NY,1
19,untapt,Talent Advocate,55000.0,NaN,NaN,0


In [157]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [158]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer()
X = c.job_title
y = c.dumsal
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [159]:
cvec = CountVectorizer(stop_words='english').fit(X_train)
df_train = pd.DataFrame(cvec.transform(X_train).todense(),
             columns=cvec.get_feature_names())

In [160]:
df_test = pd.DataFrame(cvec.transform(X_test).todense(),
                      columns=cvec.get_feature_names())
print df_train.shape
print y_train.shape
print df_test.shape
print y_test.shape

(11, 25)
(11,)
(6, 25)
(6,)


In [165]:
lr = LogisticRegression()
lr.fit(df_train, y_train)
lr.score(df_test, y_test)
#this is my title score

0.83333333333333337

In [169]:
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
cv = 3
# Perform 6-fold cross validation
scores = cross_val_score(lr, df_test, y_test, cv=cv)
print "Cross-validated scores:", scores
# Make cross validated predictions
predictions = cross_val_predict(lr, df_test, y_test, cv=cv)
accuracy = metrics.accuracy_score(y_test, predictions)
print "Cross-Predicted Accuracy:", accuracy

Cross-validated scores: [ 0.5  0.5  0.5]
Cross-Predicted Accuracy: 0.5


In [170]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(class_weight='balanced')
RF = rf.fit(df_train,y_train)
s = cross_val_score(rf, df_train, y_train, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forrest", s.mean().round(3), s.std().round(3))

Random Forrest Score:	0.556 ± 0.079


In [171]:
rf_features = pd.DataFrame(RF.feature_importances_,
                                   index = df_train.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
rf_features.head(10)

,importance
machine,0.298821
head,0.086030
digital,0.060417
data,0.059908
graduate,0.057971
science,0.054947
analysis,0.053885
success,0.053862
ai,0.050000
talent,0.047515


#### Create a few new variables in my dataframe to represent interesting features of a job title. Then I built a new Random Forest with these features. 

In [172]:
#manager, visualization, scientist, engineer, analyst, senior
c['visualization'] = map(int, c['job_title'].str.lower().str.contains('vis'))
c['manager'] = map(int, c['job_title'].str.lower().str.contains('manager')|
                  c['job_title'].str.lower().str.contains('mngr')|
                  c['job_title'].str.lower().str.contains('mgr'))
c['scientist'] = map(int, c['job_title'].str.lower().str.contains('scien'))
c['associate'] = map(int, c['job_title'].str.lower().str.contains('asso'))
c['machine'] = map(int, c['job_title'].str.lower().str.contains('mach'))
c['engineer'] = map(int, c['job_title'].str.lower().str.contains('engi')|
                         c['job_title'].str.lower().str.contains('engr'))
c['senior'] = map(int, c['job_title'].str.lower().str.contains('sr')|
                  c['job_title'].str.lower().str.contains('snr')|
                  c['job_title'].str.lower().str.contains('senior'))
#making dummies for the words at the top of this cell
c.head()

,company,job_title,avg,city,state,dumsal,visualization,manager,scientist,associate,machine,engineer,senior
2,Owkin,Legal and Business Analyst,65000.0,New York,NY,0,0,0,0,0,0,0,0
5,MINDPORTS AI,Quantitative Analysis and Machine Learning,167500.0,NaN,NaN,1,0,0,0,0,1,0,0
9,Kasisto,Machine Learning Engineer,150000.0,NaN,NaN,1,0,0,0,0,1,1,0
17,Harnham US,C++ Engineer - Computer Vision,140000.0,New York,NY,1,1,0,0,0,0,1,0
19,untapt,Talent Advocate,55000.0,NaN,NaN,0,0,0,0,0,0,0,0


In [173]:
#completing a logistic regression model
from sklearn.linear_model import LogisticRegression
X = c[['manager', 'visualization', 'scientist', 'engineer', 'senior', 'associate']]
y = c.dumsal
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)
#this is my concatonated score since it's worse I'm only going to use  

0.5

In [174]:
#and a random forest classifier
rf = RandomForestClassifier(class_weight='balanced')
s = cross_val_score(rf, X, y, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forrest", s.mean().round(3), s.std().round(3))

Random Forrest Score:	0.522 ± 0.11


In [175]:
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
# Perform 6-fold cross validation
scores = cross_val_score(rf, X, y, cv=6)
print "Cross-validated scores:", scores
# Make cross validated predictions
predictions = cross_val_predict(rf, X, y, cv=6)
accuracy = metrics.accuracy_score(y, predictions)
print "Cross-Predicted Accuracy:", accuracy

Cross-validated scores: [ 0.75  0.5   1.    0.5   0.5   0.5 ]
Cross-Predicted Accuracy: 0.588235294118


In [176]:
#sending my final to a csv
c.to_csv("~/Desktop/c.csv" , sep=',', encoding='utf-8')

In [177]:
pd.read_csv("~/Desktop/c.csv", index_col=0)

,company,job_title,avg,city,state,dumsal,visualization,manager,scientist,associate,machine,engineer,senior
2,Owkin,Legal and Business Analyst,65000.0,New York,NY,0,0,0,0,0,0,0,0
5,MINDPORTS AI,Quantitative Analysis and Machine Learning,167500.0,NaN,NaN,1,0,0,0,0,1,0,0
9,Kasisto,Machine Learning Engineer,150000.0,NaN,NaN,1,0,0,0,0,1,1,0
17,Harnham US,C++ Engineer - Computer Vision,140000.0,New York,NY,1,1,0,0,0,0,1,0
19,untapt,Talent Advocate,55000.0,NaN,NaN,0,0,0,0,0,0,0,0
29,Harnham US,Director of Data Science,250000.0,New York,NY,1,0,0,1,0,0,0,0
30,Harnham US,Head of Data Science,225000.0,New York,NY,1,0,0,1,0,0,0,0
31,TWOSENSE.AI,Senior Android Engineer,120000.0,Brooklyn,NY,0,0,0,0,0,0,1,1
33,Net-Consultants,Machine Learning Data Scientist,130000.0,NaN,NaN,1,0,0,1,0,1,0,0
35,Amenity Analytics,Graduate Training Program,60000.0,NaN,NaN,0,0,0,0,0,0,0,0
